# 4.5 Training and testing

In this step we create an Decision Tree model for our data extracted in step 3.2

### 1. Adding the libraries we need

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

### 2. Read in the feature dataset from the result of 3.2 notebook

In [2]:
directory = "../dataset-numpy/"

dataset = pd.read_csv(os.path.join(directory, '3.2 - features.csv'), delimiter=',', header=0)

dataset.describe(include='all')

,area,area_filled,area_convex,axis_major_length,axis_minor_length,eccentricity,equivalent_diameter_area,extent,feret_diameter_max,perimeter,...,h_sym_err_percent,v_sym_err_percent,pixelsum_tr,pixelsum_ml,pixelsum_mm,pixelsum_mr,pixelsum_bl,pixelsum_bm,pixelsum_br,label
count,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,...,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442.000000,1442
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ash
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162
mean,5931.861997,6299.945215,10955.404993,143.656000,103.958844,0.651493,86.403498,0.386608,151.435345,662.953378,...,0.309030,0.351024,21.240638,61.147018,60.909847,60.762136,43.341886,42.410541,41.572816,NaN
std,1264.311181,1714.453705,1662.548843,16.433003,14.106169,0.145744,9.336880,0.081937,15.457489,115.210966,...,0.081884,0.128249,8.573700,21.512239,21.496127,21.593420,25.040321,24.274897,23.637983,NaN
min,1469.000000,1469.000000,1808.000000,110.482240,14.198591,0.105603,43.247993,0.186134,120.208153,281.213203,...,0.052680,0.030307,4.000000,12.000000,14.000000,14.000000,11.000000,11.000000,11.000000,NaN
25%,4994.000000,5062.000000,9656.250000,132.065478,94.669247,0.565938,79.740569,0.325215,137.767921,561.651804,...,0.256113,0.254683,16.000000,46.000000,45.000000,45.000000,25.000000,24.000000,24.000000,NaN
50%,5944.500000,6092.500000,10486.000000,141.979676,102.872911,0.673254,86.998686,0.387129,152.200526,657.898016,...,0.311655,0.358025,19.000000,60.000000,59.500000,60.000000,34.000000,33.000000,33.000000,NaN
75%,6792.750000,7218.500000,12014.250000,153.354823,114.357259,0.755008,92.998912,0.442410,166.352036,716.079996,...,0.362123,0.452166,25.000000,76.000000,76.000000,77.000000,58.000000,56.000000,55.000000,NaN


### 3. Create training and testing datasets

We will use the `train_test_split` function from `sklearn` to split the dataset into training and testing datasets. We will use 75% of the data for training and 25% for testing.

In [3]:
X = dataset.loc[:, dataset.columns != 'label'] # Features
y = dataset.loc[:, dataset.columns == 'label'] # Target

rnd_state = 42 # Random state for reproducibility

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=rnd_state)

### 4. Find the best hyperparameters

Decision trees has a lot of hyperparameters that we can tune. We will use the `GridSearchCV` function from `sklearn` to find the best hyperparameters for our model. We will use 5-fold cross validation to find the best hyperparameters.

In [4]:
parameters = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 3, 4, 5, 6, 7, 8],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8]
}

testing_svm = tree.DecisionTreeClassifier()

clf = GridSearchCV(testing_svm, parameters, cv=5, scoring='accuracy')
clf.fit(X_train, y_train.values.ravel())

print("Best parameters set found on development set:")
print(clf.best_params_)

Best parameters set found on development set:
{'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 3, 'splitter': 'best'}


### 6. Creating and fitting model

We create and train the Decision Tree model with our training dataset without scaling.

In [5]:
# Build a model
clf = tree.DecisionTreeClassifier(**clf.best_params_)

accuracy_scores = cross_val_score(clf, X_train, y_train, cv=5)

clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3,
                       min_samples_split=3)

### 7. Making predictions

Now we can make prediction. We attempt to predict each of our testing dataset and see how well our model performs. We print out the accuracy score and the confusion matrix to see how well our model performs and what runes get confused with each other.

In [6]:
y_pred = clf.predict(X_test)

labels = ['oak', 'need', 'elk-sedge', 'spear', 'tyr', 'gift', 'bow', 'sun', 'ash', 'wealth', 'joy', 'serpent'] # used to make sure the confusion matrix is in the correct order

cm = confusion_matrix(y_test, y_pred, labels=labels)

score = accuracy_score(y_test, y_pred)*100

plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt="d", linewidths=.5, square=True, cmap='Blues_r', xticklabels=labels, yticklabels=labels)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Accuracy Score: {0}%'.format(score), size=15)
plt.show()


As we can see from the above confusion matrix, our model performs very well. The only runes that get confused with each other are `ash` and `wealth`. It also had problems with `elke-sedge`.

Now, let's see if our model is overfitting. We take the scores from using kfold on our model and checking for extreme differences between the individual accuracy scores and the standard deviation of the accuracy scores.

In [7]:
# Print the individual scores to see if there are extreme differences
for fold, score in enumerate(accuracy_scores):
    print(f"Fold {fold + 1} - Accuracy score: {score}")
print()

# Print the difference between the smallest and largest score
score_difference = max(accuracy_scores) - min(accuracy_scores)
print(f"Maximum difference between accuracy scores {score_difference:.2f}")

# Print the standard deviation to check the variability of the model
deviation = np.std(accuracy_scores)
print(f"Standard Deviation of accuracy scores: {deviation:.2f}")

Fold 1 - Accuracy score: 0.8755760368663594
Fold 2 - Accuracy score: 0.8981481481481481
Fold 3 - Accuracy score: 0.8935185185185185
Fold 4 - Accuracy score: 0.8518518518518519
Fold 5 - Accuracy score: 0.8796296296296297

Maximum difference between accuracy scores 0.05
Standard Deviation of accuracy scores: 0.02


The maximum difference between accuracy scores is 5% with a standard deviation of 2%, meaning that there likely is only very minimal overfitting.

### 8. Save the model

We binary serialize the model and save it to a file so we can use it later in our web app.

In [8]:
directory = "../classifiers/" 
path = os.path.join(directory, '4.5 - tree model.pkl')

with open(path, 'wb') as file:
    pickle.dump(clf, file)